In [1]:
import numpy as np

In [2]:
A = np.array([[1,2,3],
              [4,5,6],
              [7,8,9],
              [0,0,1]])

np.argmax(A, axis = 1) # 取行最大 index

array([2, 2, 2, 2])

In [3]:
A[-1]

array([0, 0, 1])

## **Value Iteration**

In [ ]:
# 部署算法
# V0 → V* → π*  贝尔曼 最优算子
def value_iteration(env, tol=1e-10):

    # 初始化
    v = np.zeros(env.n_states)                    # initialize value function
    q = np.zeros((env.n_states, env.n_actions))   # initialize Q-value function
    
    # 值迭代
    while True:
        v_old = np.copy(v)    

        ######## 贝尔曼-最优算子 ########

        # 更新 Q
        for a in range(env.n_actions):
            q[:, a] = env.r[:,a] + env.gamma * env.sparseT[a].dot(v)                          #00ff00
            # q = 9x4 矩阵 (代表每个(s,a)组合下的 回报期望 = 即刻回报 + 折扣未来回报)

        # 更新 V
        v = np.max(q,axis = 1)                                                                #00ff00
        # v = 9x1 向量 (选出 q 每行最大值 = 最优的 action 使得回报最大)

        # 终止条件
        if np.linalg.norm(v - v_old) < tol: 
            v_star  = v
            pi_star = np.argmax(q,axis=1)
            break
            
    return v_star, pi_star


# 检验算法
# π → V^π      贝尔曼 策略算子
def evaluate_policy_sequence(pi, env, tol=1e-10):

    v = np.zeros(env.n_states)                   # 初始化 V(s)
    q = np.zeros((env.n_states, env.n_actions))  # 初始化 Q(s,a)

    while True:

        v_old = np.copy(v) 

        ######## 贝尔曼-策略算子 ########

        # 更新 Q
        for a in range(env.n_actions):
            q[:, a] = env.r[:,a] + env.gamma * env.sparseT[a].dot(v)      # calculate Q-value #0000ff 与上面一样
            # q = 9x4 矩阵 (代表每个(s,a)组合下的 回报期望 = 即刻回报 + 折扣未来回报)

        # 更新 V
        for s in range(env.n_states):
            v[s] = q[s,pi[s]]      # calculate value function by v(s) = Q(s,pi(s))            #0000ff 与上面不一样，上面使用 max 这里使用 π(s)


        # 终止条件
        if np.linalg.norm(v - v_old) < tol: # convergence criterion
            v_pi = v
            break
                
    return v_pi

## **Policy Iteration**

In [ ]:
# 第一步 策略算子
# π_(t) → V^π, Q^π
def evaluate_policy(pi, env, tol=1e-10):

    v = np.zeros(env.n_states)
    q = np.zeros((env.n_states, env.n_actions))

    while True:

        v_old = np.copy(v)

        ######## 贝尔曼-策略算子 ########

        # 更新 Q
        for a in range(env.n_actions):
            q[:, a] = env.r[:, a] + env.gamma * env.sparseT[a].dot(v)                       #0000ff 与上面一样

        # 更新 V
        for s in range(env.n_states):
            #v[s] = q[s, :].dot(pi[s])  #TODO #0000ff 随机策略 = 概率分布
            v[s] = q[s, pi[s]]          #TODO #0000ff 固定策略 = 确定性


        # 终止条件
        if np.linalg.norm(v - v_old) < tol:
            v_pi = v
            q_pi = q
            break

    return v_pi, q_pi


# 第二步 策略优化
# Q^π → π_(t+1)
def get_greedy_policy(q_pi):
    policy = np.argmax(q_pi, axis = 1)
    return policy


# 算法部署
def policy_iteration(env, tol=1e-20):

    vs = []
    v  = np.zeros(env.n_states)
    q  = np.zeros((env.n_states, env.n_actions))
    pi = np.zeros(env.n_states, dtype=int)

    while True:
        v_old = np.copy(v)
        v, q = evaluate_policy(pi, env, tol)  # 第一步 π_(t)  →  V^π Q^π
        pi   = get_greedy_policy(q)           # 第二部 Q^π    →  π_(t+1)
        vs.append(v)

        # 终止条件
        if np.linalg.norm(v - v_old) < tol:
            break

    return vs

## 问题：

- 固定策略下，贝尔曼 策略算子 $T^\pi$ 和 最优算子 $T^*$ 的区别
- 随机策略下，贝尔曼 策略算子 $T^\pi$ 和 最优算子 $T^*$ 的区别

In [ ]:
def value_iteration(env, tol=1e-10):
    # 初始化
    v = np.zeros(env.n_states)                    # initialize value function
    q = np.zeros((env.n_states, env.n_actions))   # initialize Q-value function
    
    # 迭代
    while True:
        v_old = np.copy(v)  
    
        # 贝尔曼-最优算子
        for a in range(env.n_actions):
            q[:, a] = env.r[:,a] + env.gamma * env.sparseT[a].dot(v)  
            # q = 9x4 矩阵 (代表每个(s,a)组合下的 回报期望 = 即刻回报 + 下一步折扣期望回报)

        v = np.max(q,axis = 1)  
        # v = 9x1 向量 (选出最优的a使得回报最大 = 选出每行最大值)

        # 终止标准
        if np.linalg.norm(v - v_old) < tol: 
            v_star = v
            break
            
    return v_star
